# LC3 Notebook
_Author_: __Ramesh Yerraballi__ <br>
_Date_: __10/22/18__ <br>

This is a LC3 [Jupyter](https://jupyter.org/) notebook. We can write LC3 code directly and run it. No need for an editor and a simulator. Thanks to Doug Blank for the [Calysto LC3 Kernel](https://github.com/Calysto/calysto_lc3)

In this notebook I will explain some basic LC3 Concepts:
1. The ISA
2. Instruction Execution
3. Tour of 9 Instructions to demonstrate the workings of the language 


## LC3 ISA
The ISA (Instruction Set Architecture) for a machine is comprised of the following components:
1. The Set of Instructions supported
2. The Number of Registers both General-purpose and speical
3. The memory-layout: _address space_ and _addressability_ of the machine
4. The addressing modes supported
5. The data-types supported

The ISA is a _specification_ that is akin to a standards document that describes in detail the __what__ a machine __does__. The __how__ it does it, is called the implementation. For example, the ARM ISA describes what instructions an ARM-based machine understands, how to format them correctly and, what happens when you run them. The how is left to the manufacturer. The same ARM ISA is implemented in an iPhone from Apple, and an Android phone from (say) Samsung. One may be faster than the other but they both understand the same set of instructions. 

The LC3 ISA then, answers these five components thusly:
1. LC3 instruction set has 15 instructions: 
   1. Arithmetic-Logic: `NOT`,`AND`,`ADD`
   2. Data-Movement: Loads - `LD`, `LDR`, `LDI`; Stores - `ST`, `STR`, `STI`; `LEA`
   3. Control: `BR`, `JMP`, `JSR`, `TRAP`, `RTI`
  ![LC3 ISA](LC3-ISA.png)
  
2. LC3 has 8 general-purpose registers called $\texttt{R_0},\texttt{R_1},\texttt{R_2},\texttt{R_3},\texttt{R_4},\texttt{R_5},\texttt{R_6},\texttt{R_7}$ and, two special registers, PC (holds the location/address of the _next_ instruction), IR (holds the instruction currently executing)  and PSR (holds the status of the processor, specifically the `N`,`Z`, and `P` bits, collectively referred to as the Condition Code (`CC`) bits.
3. LC3's memory is made of $2^{16}$ locations (_address space_) of $16$ bits (_addressability_) each. You can think of memory as a table with $2^{16}$ rows and $16$ columns.
4. LC3 supports _five_ addressing modes. This tells us _where_ an operaand to an operation can be found. 
   1. Register: In a register
   2. Immediate: In the instruction itself as a signed integer
   3. PC+Offset: At a position relative to the PC
   4. Base+Offset: At a position relative to a Base-register
   5. Indirect: At an address to be found at a position relative to the PC
<a href="https://www.youtube.com/watch?feature=player_embedded&v=O4rrJieY7xc" target="_blank"><img src="https://img.youtube.com/vi/O4rrJieY7xc/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="240" height="180" border="10" /></a>
5. LC3 supports two types of data:
   1. Characters (ASCII)
   2. Signed Integers (in 2's complement)

   

## Assembler
Programmers write code in assembly language which uses mnemonics (symbols) to represent instructions. Each instruction is made up of an _operation_ and one or more _operand(s)_. The computer only understands instructions expressed in _binary_. So, an __assembler__ converts instructions written in assembly to binary that the machine understands. The binary version of the instruction is also referred to as _machine code_. The LC3 assembler allows the programmer to express their intent (the program) using _instructions_ supported by the ISA and, _pseudo-ops_, that are commands to the assembler. Pseudo-ops are easily recognized by the fact that they have a period (`.`) at their beginning. For instance, the pseudo-op, `.ORIG x3000` tells the assembler that the code that follows must be loaded to the location `x3000`.

## Instruction Execution
The execution of an individual instruction in LC3 happens in 6 stages:
1. **Instruction Fetch**: The instruction is fetched from Memory to CPU (IR register); The PC is updated
2. **Decode**: The most significant 4 bits IR[15-12] are decoded to figure out what theis instruction is.
3. **Evaluate Address**: __If__ there is an address to be computed, for example, in LD/ST, LDR/STR, LEA, LDI/STI and, BR, JSR, TRAP, JMP __Then__ the address computation is done here. 
4. **Fetch Operands**: __If__ there is an operand tthat needs to be brought in from __Memory__ __then__ this is the stage in which this happens. Only LD, LDR, LDI have an an operand fetch. Arithmetic operations get their operands from registers or the IR (in case of immediate operand), so there is no need to go to memory.
5. **Execute**: Any arithmetic/logic work and, the decision to branch or not to branch happen in this stage. 
6. **Store Result**: The writing of the result to the destination register happens in this stage.

The following video will explain the instruction execution (aka instruction cycle) in more detail:
<a href="https://www.youtube.com/watch?feature=player_embedded&v=FvHzVspAq5o" target="_blank"><img src="https://img.youtube.com/vi/FvHzVspAq5o/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="240" height="180" border="10" /></a>

## Instruction Tour
Lets take an look at what the LC3 machine's state looks like 

In [1]:
%reset


Welcome to the LC-3 simulator.

The contents of the LC-3 tools distribution, including sources, management
tools, and data, are Copyright (c) 2003 Steven S. Lumetta.

The LC-3 tools distribution is free software covered by the GNU General
Public License, and you are welcome to modify it and/or distribute copies
of it under certain conditions.  The file COPYING (distributed with the
tools) specifies those conditions.  There is absolutely no warranty for
the LC-3 tools distribution, as described in the file NO_WARRANTY (also
distributed with the tools).

Have fun.

Registers:
PC: x3000
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


As can be seen from above, the 8 registers(`R0-R7`) the PC and the Condition Code bits which are the lower 3-bits of the PSR, make up the state of a machine. We note that the PC value is set by default to `x3000`, but this can be changed by the `.ORIG` pseudo-op like so:

In [2]:
.ORIG x2500

Assembled! Use %dis or %dump to examine; use %exe to run.


In [4]:
%regs


Registers:
PC: x2500
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


### Nine Instructions
Rather than go through each of the instructions that LC3 supports I will cover 9 of the 15 instructions that you need, to get almost anything done. These nine instructions are: `NOT`,`AND`, `ADD`, `LEA`, `LD`, `LDR`, `ST`, `STR`, `BR` and `TRAP`. We will also look at the following pseudo-ops: `.ORIG`, `.FILL` `.BLKW` and `.END`. Pseudo-ops are not instructions in the ISA but simply commands to give to the assembler (which converts our assembly to machine code) to tell it to do certain things.

First, I'll take one simple instruction, the `ADD` to explain the relationship between assembly and machine code. Lets take the following instance of the ADD instrcution, which says to add the contents of registers `R2` and `R4` and put the result in `R1`:

In [5]:
ADD R1, R2, R4

Assembled! Use %dis or %dump to examine; use %exe to run.


In [6]:
%dump

Memory dump:
           x2500: x1284


The machine code produced by the assembler for this instruction has four parts:
1. The opcode for ADD is `0001` 
2. The destination register DR is R1: `001`
3. The first source register SR1 is R2: `010`
4. The first source register SR2 is R4: `100`

From the cell output above we see the machine code is: `0001001010000100`, which is `x1284` in hex. 

In [7]:
%dis

Memory disassembled:
           x2500: x1284  ADD R1, R2, R4                            [line: 1]


Lets run the one `ADD` instruction at `x2500`; First we make sure the PC is x2500

In [8]:
%pc x2500


Registers:
PC: x2500
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


Lets set R2, R4 to some initial values, say `2` and `-3` respectively

In [9]:
%reg 2 x0002


Registers:
PC: x2500
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0002 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


-3 in 16-bit 2's complement is `NOT(0000 0000 0000 0011)+1` = `0xFFFD`

In [10]:
%reg 4 xFFFD


Registers:
PC: x2500
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0002 R3: x0000 
R4: xFFFD R5: x0000 R6: x0000 R7: x0000 


Lets execute the single instruction at `x2500`:

In [11]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x2501
(1/6) ADD R1, R2, R4 [1] (x2501*: x1284)
    R1 <= xFFFF
    NZP <= (1, 0, 0)

Registers:
PC: x2501
N: 1 Z: 0 P: 0 
R0: x0000 R1: xFFFF R2: x0002 R3: x0000 
R4: xFFFD R5: x0000 R6: x0000 R7: x0000 


We note the result in R1 is `xFFFF`(`1111111111111111`) which is `-1`

Now, we will look at a program with all nine instructions I wish to cover:

In [12]:
%reset


Welcome to the LC-3 simulator.

The contents of the LC-3 tools distribution, including sources, management
tools, and data, are Copyright (c) 2003 Steven S. Lumetta.

The LC-3 tools distribution is free software covered by the GNU General
Public License, and you are welcome to modify it and/or distribute copies
of it under certain conditions.  The file COPYING (distributed with the
tools) specifies those conditions.  There is absolutely no warranty for
the LC-3 tools distribution, as described in the file NO_WARRANTY (also
distributed with the tools).

Have fun.

Registers:
PC: x3000
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


In [13]:
;*******************************
;   LC3 in 10 minutes
; 9 instructions you need to know to 
; get most programs done.
    .ORIG x3000
    AND   R0,R0,#0    ; clear R0
    ADD   R1,R0,#15   ; R1 <- 15
    ADD   R2,R1,R1    ; R2 <- 15+15 (30)
    ADD   R2,R2,R1    ; R2 <- 30+15 (45)
    ADD   R2,R2,#-3   ; R2 <- 45-3 = 42
    NOT   R2,R2       
    ADD   R2,R2,#1    ; R2 <- -42
    LD    R1,xB       ; R1 <- Mem[x3008+xA] which is xABCD
    LEA   R3,Input    ; R3 <- x3009+x9 = x3012 (address of Input)
    LDR   R4,R3,#0    ; Another way to get Mem[x3012]
    ADD   R0,R1,R2    ; Comparing R1(xABCD) and R2(-42)
    BRz   IsStar      ; Equal means R1 is 42 and Z must be set
    LD    R0,NotStar  ; Here means Input is not the character '*',so R0 <- x2D (ascii code for character '-')
    BR    x1          ; Branch to call TRAP
IsStar  ADD   R0,R1,#0 ; It is a '*', so copy it to R0
    TRAP  x21         ; Call TRAP to print character in R0 to console
    ST    R0,Output   ; Store R0 to the location labeled Output 
    STR   R0,R3,#1    ; Does the same as above because R3 is the address x3012
        TRAP  x25     
Input   .FILL 42      
Output  .BLKW 1
NotStar .FILL x2D
    .END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [14]:
%dis

Memory disassembled:
           x3000: x5020  AND R0, R0, #0                            [line: 1]
           x3001: x122F  ADD R1, R0, #15                           [line: 2]
           x3002: x1441  ADD R2, R1, R1                            [line: 3]
           x3003: x1481  ADD R2, R2, R1                            [line: 4]
           x3004: x14BD  ADD R2, R2, #-3                           [line: 5]
           x3005: x94BF  NOT R2, R2                                [line: 6]
           x3006: x14A1  ADD R2, R2, #1                            [line: 7]
           x3007: x220B  LD R1, INPUT                              [line: 8]
           x3008: xE60A  LEA R3, INPUT                             [line: 9]
           x3009: x68C0  LDR R4, R3, 0                             [line: 10]
           x300A: x1042  ADD R0, R1, R2                            [line: 11]
           x300B: x0402  BRz ISSTAR (or 2)                         [line: 12]
           x300C: x2008  LD R0, NOTSTAR             

#### Single-Stepping
Lets single-step through this program so we can see the working of each instruction in turn. First we will  set the pc to x3000:

In [15]:
%pc x3000


Registers:
PC: x3000
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


In [16]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3001
(1/6) AND R0, R0, #0 [1] (x3001*: x5020)
    R0 <= x0000
    NZP <= (0, 1, 0)

Registers:
PC: x3001
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


Lets see what is being displayed on executing the `AND` instruction at `x3000`:
1. the `PC` has been incremented by 1 to `x3001`
2. the specific `AND R0, R0,#1` instruction's machine code is `x5020`
3. executing the instruction took 6 cycles; 1/6 is the _fetch instruction_ phase during which, the PC was updated to x3001.
4. `R0` got a resulting value of `x0000`
5. the CC bits have been modified as, `N=0`,`Z=1`, `P=0`

The purpose of the next four instructions together is to get the number 42 into register `R2`. The largest number I can get into a register using the `ADD` immediate instruction is 15 (5-bit signed integer max). So, I will add 15 three times and subtract -3. I choose this sequence so we can see how:
1. you can add either two registers and put the result in a destination register:<br>
     `ADD R2,R1,R1` and `ADD R2, R2, R1`
2. add a register and a signed 5-bit integer and put the result in a destination register:<br>
     `ADD R1, R0, #15` and `ADD R2, R2, #-3`
3. the destination could be one of the sources:<br>
     `ADD R2, R2, R1`
4. the destination need not always be one of the sources:<br>
     `ADD R2,R1,R1`

In [17]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3002
(2/12) ADD R1, R0, #15 [2] (x3002*: x122F)
    R1 <= x000F
    NZP <= (0, 0, 1)

Registers:
PC: x3002
N: 0 Z: 0 P: 1 
R0: x0000 R1: x000F R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


`R1` has x000F (15)

In [18]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3003
(3/18) ADD R2, R1, R1 [3] (x3003*: x1441)
    R2 <= x001E
    NZP <= (0, 0, 1)

Registers:
PC: x3003
N: 0 Z: 0 P: 1 
R0: x0000 R1: x000F R2: x001E R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


`R2` has x001E (30)

In [19]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3004
(4/24) ADD R2, R2, R1 [4] (x3004*: x1481)
    R2 <= x002D
    NZP <= (0, 0, 1)

Registers:
PC: x3004
N: 0 Z: 0 P: 1 
R0: x0000 R1: x000F R2: x002D R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


`R2` has x002D (45)

In [20]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3005
(5/30) ADD R2, R2, #-3 [5] (x3005*: x14BD)
    R2 <= x002A
    NZP <= (0, 0, 1)

Registers:
PC: x3005
N: 0 Z: 0 P: 1 
R0: x0000 R1: x000F R2: x002A R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


End of the 4-steps: `R2` has the value `x002A` (42)

The next two instructions convert R2 to -42 by taking its 2's complement: `NOT and add 1`

In [21]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3006
(6/36) NOT R2, R2 [6] (x3006*: x94BF)
    R2 <= xFFD5
    NZP <= (1, 0, 0)

Registers:
PC: x3006
N: 1 Z: 0 P: 0 
R0: x0000 R1: x000F R2: xFFD5 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


In [22]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3007
(7/42) ADD R2, R2, #1 [7] (x3007*: x14A1)
    R2 <= xFFD6
    NZP <= (1, 0, 0)

Registers:
PC: x3007
N: 1 Z: 0 P: 0 
R0: x0000 R1: x000F R2: xFFD6 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


`R2` has the value `xFFD6`, which is decimal -42.

__We are now at x3007 as indicated by the PC. We can look at the code there__:

In [23]:
%dis x3007

Memory disassembled:
           x3007: x220B  LD R1, INPUT                              [line: 8]
           x3008: xE60A  LEA R3, INPUT                             [line: 9]
           x3009: x68C0  LDR R4, R3, 0                             [line: 10]
           x300A: x1042  ADD R0, R1, R2                            [line: 11]
           x300B: x0402  BRz ISSTAR (or 2)                         [line: 12]
           x300C: x2008  LD R0, NOTSTAR                            [line: 13]
           x300D: x0E01  BRnzp x300F (or 1)                        [line: 14]
ISSTAR:    x300E: x1060  ADD R0, R1, #0                            [line: 15]
           x300F: xF021  OUT                                       [line: 16]
           x3010: x3003  ST R0, OUTPUT                             [line: 17]
           x3011: x70C1  STR R0, R3, 1                             [line: 18]
           x3012: xF025  HALT                                      [line: 19]
INPUT:     x3013: x002A  NOOP - (no BR to x30

The instruction we are about to execute is a _load_ instruction with a PC-Offset operand. That is, we specify what to load from memory by giving its position relative to the location of the current instruction. We can do this either directly by giving a label to the location we want to load from, or, by specifying the offset which is what we did in this case:<br>
    `LD    R1,xB` <br>
This would have been the same as saying:<br>
    `LD    R1,Input` <br>
The first version tells the assembler explictly what the offset is (`xB`) to get to `Input`. The second version asks the assembler to figure out what the offset is to get to `Input`. In both cases the machine code generated is the same:<br>
    `x220B` which is `0010 001 000001011` (added spaces for clarity) <br>

Lets run the instruction:

In [24]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3008
(8/52) LD R1, INPUT [8] (x3008*: x220B)
  Reading memory[x3013] (x002a) =>
    R1 <= x002A
    NZP <= (0, 0, 1)

Registers:
PC: x3008
N: 0 Z: 0 P: 1 
R0: x0000 R1: x002A R2: xFFD6 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


The next two instrcutions acheive the same thing as the previous `LD` instruction. The `LEA` instruction gets the address of Input into register `R3` (`x3013`). The `LDR` instruction then uses the address in R3 as the __base__ (instead of the PC) and adds an __offset__ (of 0) to that address to fetch the contents of memory location `x3013`, putting that (`xABCD`) intor `R4`. 

So, what is the point of doing something in 2 instructions when you could have done it in 1 instruction?<br>
The answer depends on whether the programmer has any use for the address later. If she does, then this extra step now may save us a step later. Also, the `LDR` is able to access not just the contents of the location whose address is in `R3` but any location that can be specified by an offset relative to `R3`. As you will see in a later instruction, `STR R0,R3,1`, we are able to write a value to the `Output` by using `R3` as the __base__ and using an __offset__ of 1.

For more details on how LD, LEA and LDR work, please watch the following video:

<a href="https://www.youtube.com/watch?feature=player_embedded&v=c0fmQvw0oE4" target="_blank"><img src="https://img.youtube.com/vi/c0fmQvw0oE4/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="240" height="180" border="10" /></a>


In [25]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x3009
(9/58) LEA R3, INPUT [9] (x3009*: xE60A)
    R3 <= x3013
    NZP <= (0, 0, 1)

Registers:
PC: x3009
N: 0 Z: 0 P: 1 
R0: x0000 R1: x002A R2: xFFD6 R3: x3013 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


In [26]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x300A
(10/67) LDR R4, R3, 0 [10] (x300A*: x68C0)
  Reading memory[x3013] (x002a) =>
    R4 <= x002A
    NZP <= (0, 0, 1)

Registers:
PC: x300A
N: 0 Z: 0 P: 1 
R0: x0000 R1: x002A R2: xFFD6 R3: x3013 
R4: x002A R5: x0000 R6: x0000 R7: x0000 


## Changing the flow
So far in our example, the PC got incremented by one on each instruction execution. This means we are executing the code _sequentially_ stepping one instruction after an other. All programming languages support the ability to _change_ the flow, i.e., go _forward_ a block ($\geq 1$) instructions or go _back_ to a point before the current instruction based on a _condition_. This is called a __conditional__ statement. After executing a condition statement, the `PC` (recall it holds the address of the next instruction to execute) can either, just `PC+1` or `PC+<Offset_to_the_point_forward_or_backward_to_go>`, depending on whether the condition is `false` or `true`. 

The condition itself is expressed in the form of a check of the three condition code(`CC`) bits `N`, `Z` and `P`, which indicate whether the "__most recent__" operation resulted in a `negative` value, a `zero` value or, a `positive` value respectively. As, not all instructions modify these bits, the check is really based on the "__most recent condition code modifying__" instruction. In the LC3 instruction list above, all instructions that modify the CC bits are indicated with a `+` on them. Here is that list: `ADD`,`AND`, `NOT`,`LD`, `LDR`, `LDI` and `LEA`.   

In the example here, we want to see __if__ the `Input` (which is currently in both registers `R1` and `R4`) is equal or not equal to `42`. So, we take the difference between `R1` and `-42`(which is in `R2`) and test whether the result is a __zero__. That is, if the `Z` bit is set. If it is indeed set then we know `Input` is `42` (which by the way is the ASCII character `*`: star). So, we branch to a location labeled `IsStar`, i.e., set `PC` to the address of `IsStar`. Note, that if the condition check fails then `PC` simply stays the course (i.e., `PC+1`). In this example, the Input is `42`, so the result of the ADD is a zero and the branch will be taken, with the new value of `PC` being `x300E`. 

In [27]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x300B
(11/73) ADD R0, R1, R2 [11] (x300B*: x1042)
    R0 <= x0000
    NZP <= (0, 1, 0)

Registers:
PC: x300B
N: 0 Z: 1 P: 0 
R0: x0000 R1: x002A R2: xFFD6 R3: x3013 
R4: x002A R5: x0000 R6: x0000 R7: x0000 


In [28]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x300C
(12/79) BRz ISSTAR (or 2) [12] (x300C*: x0402)
    PC <= x300E
    True - branching to x300E

Registers:
PC: x300E
N: 0 Z: 1 P: 0 
R0: x0000 R1: x002A R2: xFFD6 R3: x3013 
R4: x002A R5: x0000 R6: x0000 R7: x0000 


The output above (`True - branching to x300E`) shows that the condition evaluated to True and the branch is taking us to `x300E`, the new value of the PC. 

## Invoking OS services
Thus far in our example, we have been looked at instructions that do arithmetic/logic, load, or branch. None of them produce any output that can be displayed, say on the screen (called console). We will now use a service provided by the LC3 operating system that does exactly that - Displays an output (in this case a character) on the console. Here, the console is the same place our cell output is getting displayed. 

OS services are invoked by the running the `TRAP` instruction. The LC3 OS currently provides the following services 6 services which are invoked by passing to TRAP an operand whose value is the `Trap Vector`.
![TRAPs](Traps.png)

For example, to invoke a service that outputs a single character to the console the Trap Vector we use is `x21`. The trap service expects that the character you want displayed will be put in register `R0`. So, for example, assuming the register `R0` has the hex value `x2A` (decimal 42, which is the ASCII code for the character `*`) then, the service will display a `*` on the console. We see from the table above that there are assembly shortcuts (`OUT` for `TRAP x21`) for each TRAP service, so one does not have to memorize the Trap Vector values.

In our code, the value `42` is in register `R1` and not in `R0`. So we first copy R1 to R0 and then invoke the TRAP service. Here, I dont want to step through the service routine so I set a break-point at the instruction after `TRAP x21` at address `x3010` and continue the execution:

In [29]:
%bp x3010

Breakpoints
    1)            x3010: x3003  ST R0, OUTPUT                             [line: 17]


In [30]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x300F
(13/85) ADD R0, R1, #0 [15] (x300F*: x1060)
    R0 <= x002A
    NZP <= (0, 0, 1)

Registers:
PC: x300F
N: 0 Z: 0 P: 1 
R0: x002A R1: x002A R2: xFFD6 R3: x3013 
R4: x002A R5: x0000 R6: x0000 R7: x0000 


In [31]:
%cont

*...breakpoint hit at x3010
Computation SUSPENDED
Instructions: 20
Cycles: 149 (0.000074 milliseconds)

Registers:
PC: x3010
N: 0 Z: 0 P: 1 
R0: x002A R1: x002A R2: xFFD6 R3: x3013 
R4: x002A R5: x0000 R6: x0000 R7: x3010 


We note the `*` displayed above as `TRAP 21` executed with `R0` holding the ASCII code `x2A`. We can also see the console output anytime we want as seen below

In [32]:
%console


Console:
*


## Storing works like Loading
We saw how LD and LDR work above. They load data from memory to the CPU. The instructions to store data from the CPU to the memory are, ST and STR which work the same way as far as their operands go, except direction of data movement. 

In [33]:
%dis x3010

Memory disassembled:
           x3010: x3003  ST R0, OUTPUT                             [line: 17]
           x3011: x70C1  STR R0, R3, 1                             [line: 18]
           x3012: xF025  HALT                                      [line: 19]
INPUT:     x3013: x002A  NOOP - (no BR to x303E) (or 42, '*')      [line: 20]
OUTPUT:    x3014: x0000  NOOP - (no BR to NOTSTAR) (or 0)          [line: 21]
NOTSTAR:   x3015: x002D  NOOP - (no BR to x3043) (or 45, '-')      [line: 22]


The next two instructions: <br>
> x3010: x300__3__  ST R0, OUTPUT <br>
> x3011: x70C1  STR R0, R3, __1__   <br>                   

have exactly the same end result, they store the contents of R0 (which is 42) to the `OUTPUT`.
1. The `ST` instruction does it by using the `PC+Offset` addressing mode, where the offset from x3011 is, __3__ to get to `OUTPUT` at `x3014`
2. The `STR` instruction does it by using the `Base+Offset` addressing mode, where the offset from the Base register (`R3`) holding the address `x3013` is, __1__ to get to `OUTPUT` at `x3014`

## The End
The last instruction we execute is an OS service to shutdown/halt the machine. This is the `TRAP x25` instruction which is displayed by its shortcut (`HALT`) in the listing above. 

In [34]:
%cont

Computation completed
Instructions: 23
Cycles: 177 (0.000088 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x002A R1: x002A R2: xFFD6 R3: x3013 
R4: x002A R5: x0000 R6: x0000 R7: x3013 


`The End`